<a href="https://colab.research.google.com/github/SwastikGorai/under_construction_projects/blob/master/asl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
!pip install tensorflow-gpu==2.4.0
!pip install kaggle
!pip install opencv-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.8 MB 39 kB/s 
     |████████████████████████████████| 3.8 MB 63.6 MB/s 
     |████████████████████████████████| 14.9 MB 65.1 MB/s 
     |████████████████████████████████| 132 kB 71.0 MB/s 
     |████████████████████████████████| 462 kB 74.2 MB/s 
     |████████████████████████████████| 2.9 MB 60.1 MB/s 
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=2d321c67f31c3ad79ebc44afcce5590d027b8bacccf5b10c1299ff1569e6c32c
  Stored in directory: /root/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-linux_x86_64.whl size=72340 sha256=ec93e1e9912f549f599cbc1bf89c76bdc18198f659389bbc15741742f5f24184
  Stored in directory: /root/.cache/pip/wheels/5f/fd/9e/b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
Successfully built termc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Replace "YOUR_API_KEY" with your actual API key
!echo '{"username":"swastikgorai","key":"07b47c761e08b4334725001f4d5287b2"}' > /root/.kaggle/kaggle.json

# Download the ASL dataset
!kaggle datasets download -d grassknoted/asl-alphabet

# Unzip the downloaded dataset
!unzip asl-alphabet.zip


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os

# Load the data
X = []
y = []

# Load the images and labels from the dataset
for directory in os.listdir('asl_alphabet_train'):
  for filename in os.listdir(f'asl_alphabet_train/{directory}'):
    for ff in os.listdir(f'asl_alphabet_train/{directory}/{filename}'):
      image = cv2.imread(f'asl_alphabet_train/{directory}/{filename}/{ff}')
      print(image)
      if image is not None:
        image = cv2.resize(image, (64, 64))
        X.append(image)
        y.append(directory)

In [8]:
# Convert the data to NumPy arrays
X = np.array(X)
y = np.array(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train, X_val, y_train, y_val)

In [9]:
# Define the model
model = tf.keras.models.Sequential()

# Add the first convolutional layer
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Add the second convolutional layer
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Add the third convolutional layer
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Add a flattening layer
model.add(tf.keras.layers.Flatten())


In [10]:
# Add the first fully connected layer
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))

# Add the second fully connected layer
model.add(tf.keras.layers.Dense(26, activation='softmax'))

In [11]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
from sklearn.preprocessing import LabelEncoder

# Encode the labels
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.transform(y_val)


In [15]:
model.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f46505d38b0>>

In [13]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
2175/2175 [==============================] - 23s 7ms/step - loss: 0.0702 - accuracy: 0.9995 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
2175/2175 [==============================] - 14s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
2175/2175 [==============================] - 14s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
2175/2175 [==============================] - 15s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
2175/2175 [==============================] - 14s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
2175/2175 [==============================] - 15s 7ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
2175/2175 [==============================] - 15s 7ms/step - loss: 0.0000e+0

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)

print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')


In [15]:
# Save the model
model.save('model.h5')

In [ ]:
# Save the model in the SavedModel format
tf.saved_model.save(model, 'saved_model')

In [ ]:
!tflite_convert \
  --saved_model_dir=saved_model \
  --output_file=model.tflite

